### Задача
Нужно защитить данные клиентов страховой компании. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## 1. Загрузка данных

In [16]:
df = pd.read_csv('/datasets/insurance.csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

In [17]:
matrix_features = df.drop('Страховые выплаты', axis=1)

In [18]:
features = np.array(matrix_features)
matrix_f1 = matrix_features.values

In [19]:
state = np.random.RandomState(12345)
P = state.normal(size = (4, 4))
P1 = np.linalg.inv(P)

Умножаем матрицу признаков X на матрицу-шифровальщик P.

In [20]:
M = features.dot(P)

In [21]:
matrix_target = df['Страховые выплаты']
target = np.array(matrix_target)

In [22]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
mse = mean_squared_error(target, predictions)
print("mse:", mse)

mse: 0.1233468894171086


In [23]:
model = LinearRegression()
model.fit(M, target)
predictions = model.predict(M)
mse = mean_squared_error(target, predictions)
print("mse:", mse)

mse: 0.12334688941710838


Веса модели W и W0 находятся в прямой зависимости от признаков X. Если изменяется на какую-то величину совокупность признаков соответственно должны и измениться параметры модели. Модель чёрного ящика. На входе признаки, чёрный ящик - модель обучения, на выходе параметры (веса). На входе изменения, на выходе тоже должны быть соответственно.

$$
w (X^T X) =  X^T y
$$

$$
w' = ((PX)^T PX)^{-1} (PX)^T y = P^{-1} (X^T X)^{-1} (P^{T})^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^{T})^{-1} P^T w (X^T X)
$$

Сокращаем матрицы $P^T и (P^{T})^{-1}, (X^T X)^{-1} и (X^T X)$

$$
w' = P^{-1} w
$$


$$
a' = P X P^{-1} w
$$
$$P  P^{-1} = I $$
$$
a' = X w
$$

$$
a = a'
$$

 Нет, качество почти не поменяется. Разница a и a' и соответственно MSE(a,y) и MSE(a',y) стремиться к нулю.

## 3. Алгоритм преобразования

1) Таблицу (датасет) с данными преобразуем в матрицу.
2) Умножаем матрицу с данными на обратимую матрицу-ключ, для шифрования данных датасета. Получаем зашифрованную матрицу с данными.

## 4. Проверка алгоритма

Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [24]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
R2 = r2_score(target, predictions)
print("R2:", R2)

R2: 0.42494550286668


In [25]:
model = LinearRegression()
model.fit(M, target)
predictions = model.predict(M)
R2 = r2_score(target, predictions)
print("R2:", R2)

R2: 0.4249455028666811


Метрика качества R2 моделей до и после преобразования практически не отличается.